### This is a demo of AssemblyAI API and Dalle API

In [1]:
import replicate
import streamlit as st
import urllib.request
import webbrowser 
import os
import requests
from time import sleep

In [18]:
# Declaring streamlit horizontal containers
st_header=st.beta_container()
audio_source=st.beta_container()
sum_gen=st.beta_container()
stable_dif=st.beta_container()
use_case=st.beta_container()
chapter_sum=st.beta_container()

2022-10-26 22:42:52.122 
  command:

    streamlit run /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
with st_header:
    st.title('AI Draws a Podcast')

In [19]:
# global variables
base_endpoint = 'https://api.assemblyai.com/v2'
headers = {'authorization': os.environ['api_token']}

In [20]:
# function to send audio to assemblyai
def post_chapters(audio_url):
    payload = {"audio_url": audio_url,
               "auto_chapters": True,}
    response = requests.post(f'{base_endpoint}/transcript',
                             json=payload,
                             headers=headers)
    return response.json()

# function to poll assemblyai for transcript
def get(transcript_id, status):
    current_status = status
    while current_status != 'completed':
        response = requests.get(f'{base_endpoint}/transcript/{transcript_id}', 
                                headers=headers)
        current_status = response.json()['status']
        print(f"Current Status: {current_status}...")
        sleep(3)
    return response.json()

In [4]:
post_response = post(audio_url='https://www.listennotes.com/e/p/bd7844463bac4decaf480a63b121c94d/')
print('processing:', post_response['id'])
get_response = get(transcript_id=post_response['id'],
                       status=post_response['status'])

processing: rzwg2e1g43-2f1c-45e0-b234-e997d034d7f1
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: completed...


In [5]:
print(f"Summary: {get_response.get('summary')}")

Summary: Today. Michael Pollakar. And you are going to talk about financial independence. Michael and you don't necessarily agree on what retirement means. You don't need 70% of your current income to retire, but you need to do more in the early years to pay off your mortgage.


In [13]:
# Instantiate stable diffusion model from replicate
model = replicate.models.get("stability-ai/stable-diffusion")
prefix = "Drawing of "
output = model.predict(prompt=prefix + get_response.get('summary'))

In [14]:
    webbrowser.open(output[0], new=0, autoraise=True)

True

In [25]:
post_chapters_response = post_chapters(audio_url='https://www.listennotes.com/e/p/bd7844463bac4decaf480a63b121c94d/')
print('processing:', post_chapters_response['id'])
get_chapters_response = get(transcript_id=post_chapters_response['id'],
                                            status=post_chapters_response['status'])
max_chapters = 5  # Limiting chapter count so we don't spend a really long time generating images
chapter_count = 0

processing: rskm9q3iry-47bc-4b0b-9aa2-1aa30760a39b
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: processing...
Current Status: completed...


AttributeError: 'list' object has no attribute 'items'

In [26]:
for chapter_info in get_chapters_response.get("chapters"):
    if chapter_count < max_chapters:
        output = model.predict(prompt=prefix + chapter_info.get('summary'))
        webbrowser.open(output[0], new=0, autoraise=True)
    else:
        print("Exceeded max chapter count")